# MVD 7. cvičení

## 1. část - Vytvoření dat

V dnešním cvičení je před implementací shlukování potřeba vygenerovat vlastní data. Cílem je vytvořit několik datasetů dle předlohy a vizualizovat je. Postačí dosažení podobného pozice a rozložení bodů.

In [2]:
from sklearn.datasets import make_blobs, make_moons, make_circles
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import math
from plotly.subplots import make_subplots

### 1. dataset - 2 třídy, 100 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset1.png)

Připravený dataset pro další úlohu (k-means) může vypadat například takto:
![prepared](expected_outputs/prepared_dataset.png)

In [3]:
def print_dataset(df, color = 'labels'):
    fig = px.scatter(df, y='y', x='x', color=color)
    fig.show()

In [4]:
def print_subplots(df, centroids):
    fig = make_subplots(rows=2, cols=1,
                        subplot_titles=("Original", "My func"))
    plot1 = px.scatter(df, x='x', y='y', color='labels')
    plot2 = px.scatter(df, x='x', y='y', color='KMEANS')
    fig.add_traces(plot1.data, rows=1, cols=1)
    fig.add_traces(plot2.data, rows=2, cols=1)
    fig.add_traces(go.Scatter(
        x=centroids['x'],
        y=centroids['y'],
        mode='markers',
        marker=dict(
            size=10,
            line=dict(width=3, color='pink'),
            symbol=[114]
        )
    ), rows=2, cols=1)

    fig.update_layout(height=1000, width=1000,
                      legend_tracegroupgap=180,
                      showlegend=False,
                      title_text="Kmean")
    fig.show()

In [16]:
data,labels = make_blobs(n_samples=100,n_features=2,centers=2)
df1 = pd.DataFrame(data, columns=['x', 'y'])
df1['labels'] = labels.astype('str')
df1['KMEANS'] = ""
df1['dist'] = ""
print_dataset(df1)

### 2. dataset - 4 třídy, 200 datových bodů
Použijte funkci ```make_blobs``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html).

![dataset1](expected_outputs/dataset2.png)

In [17]:
data,labels = make_blobs(n_samples=200,n_features=2,centers=4)
df2 = pd.DataFrame(data, columns=['x', 'y'])
df2['labels'] = labels.astype('str')
df2['KMEANS'] = ""
df2['dist'] = ""
print_dataset(df2)

### 3. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_moons``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html#sklearn.datasets.make_moons).

![dataset1](expected_outputs/dataset3.png)

In [18]:
data,labels = make_moons(n_samples=200,noise=.1)
df3 = pd.DataFrame(data, columns=['x', 'y'])
df3['labels'] = labels.astype('str')
df3['KMEANS'] = ""
df3['dist'] = ""
#print_dataset(df3)

### 4. dataset - 2 třídy, 200 datových bodů
Použijte funkci ```make_circles``` [odkaz na dokumentaci](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html#sklearn.datasets.make_circles).

![dataset1](expected_outputs/dataset4.png)

In [19]:
data,labels = make_circles(n_samples=200,noise=.1,factor=.2)
df4 = pd.DataFrame(data, columns=['x', 'y'])
df4['labels'] = labels.astype('str')
df4['KMEANS'] = ""
df4['dist'] = ""
#print_dataset(df4)

## 2. část - K-means

Implementujte K-means algoritmus, aplikujte ho na všechny 4 datasety a zobrazte výsledky algoritmu. Zároveň se pokuste zvýraznit centroid ve vizualizaci výsledku. Pro aplikaci K-means můžete použít počet tříd v každém datasetu (není třeba hledat parametr K automaticky).

In [20]:
def print_centroids(df, centroids):
    fig = px.scatter(df, y='y', x='x', color='labels')
    fig.add_trace(go.Scatter(
        x=centroids['x'],
        y=centroids['y'],
        mode='markers',
        marker=dict(
            size=10,
            line=dict(width=3, color='pink'),
            symbol=[114]
        )
    ))
    fig.update_layout(coloraxis_showscale=False)
    fig.show()

In [21]:
def euq_distance(point, other):
    return np.sqrt((other['x'].values-point['x'])**2 + (other['y'].values-point['y'])**2)

def new_centroid(points):
    dist= np.zeros(len(points))
    for idx,point in points.iterrows():
        dist[idx] = np.sum(euq_distance(point, points))
    idx_min_dist = np.argmin(dist)
    return points.iloc[idx_min_dist]

In [57]:
def k_means(df,num_labels, graph = False):
    centroids = df.sample(n=num_labels).reset_index()
    distances = np.zeros((len(df),num_labels)) #každý sloupec jeden centroid  
    for i in range(50):
        for idx,centroid in centroids.iterrows():    
            distances[:,idx] = euq_distance(centroid, df)
        min_dist_idx = np.argmin(distances, axis=1)
        centroids.drop(centroids.index, inplace=True)
        for label_idx in np.unique(min_dist_idx):
            idx = np.where(min_dist_idx == label_idx)[0]
            points = df.iloc[idx].reset_index()
            df.loc[idx,'KMEANS'] = label_idx
            df.loc[idx,'dist'] = distances[idx,label_idx]
            centroids.loc[len(centroids.index)] = new_centroid(points)  
    if graph:
        print_subplots(df, centroids)
    return centroids


In [58]:
datasets = [df1, df2, df3, df4]
for df in datasets:
    k_means(df, len(df['labels'].unique()), graph=False)

## Bonus - K-means s nalezenou hodnotou K

Zkuste najít algoritmy, které umožňují hledat hodnotu K pro K-means. Implementujte alespoň jeden z nich a ověřte, že funguje. Ověření proveďte nejen na našich předchozích datasetech, ale zkuste si vygenerovat i dataset s větším množstvím tříd (10-20). (1 bod)

Za implementaci více algoritmů, případně za hlubší analýzu výsledků (např. kdy už přestává hledání hodnoty K správně fungovat) je možné získat ještě jeden bonusový bod.

In [46]:
#### ELBOW METOD ###
def elbow(df, kmax):
    sum_dist = np.zeros(kmax)
    for k in range(1,kmax+1):
        centroids = k_means(df, k, graph = False)
        sum_dist[k-1]= df['dist'].sum()
    return sum_dist

In [61]:
datasets = [df1, df2, df3, df4]
for idx,df in enumerate(datasets):
    kmax = 10
    dist = elbow(df.copy(), kmax=kmax)
    correct_k = len(df['labels'].unique())
    df = pd.DataFrame(dict(
        sum_dist = dist,
        k = range(1,kmax+1)
    ))
    fig = px.line(df,x='k',y='sum_dist', title='df' + str(idx+1) + ", correct k: " + str(correct_k))
    fig.show()